# **Proyecto 2** | Modelos de Regresión Lineal

### Link del Repositorio

* https://github.com/paulabaal12/PROY3-DM

### Integrantes del equipo
- [Mónica Salvatierra - 22249](https://github.com/alee2602)
- [Paula Barillas - 22764](https://github.com/paulabaal12)
- [Derek Arreaga - 22537](https://github.com/FabianKel) 
- [Juan Pablo Solís - 22102](https://github.com/JPS4321)

## **Propuesta Inicial**

### Planteamiento del Problema

bla bla bla bla

### Metodología Propuesta

bla bla bla bla

### Resultados Esperados

bla bla bla bla

## **Análisis Exploratorio de Datos**

### Importar las librerías a utilizar

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importar datos

Primero se importan los datos en data frames individuales para analizar los campos de cada archivo antes de juntarlos

In [26]:
data_path = 'data/'
df_lists = []

customers_df = pd.read_csv(data_path + 'olist_customers_dataset.csv')
geolocation_df = pd.read_csv(data_path + 'olist_geolocation_dataset.csv')
order_items_df = pd.read_csv(data_path + 'olist_order_items_dataset.csv')
order_payments_df = pd.read_csv(data_path + 'olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv(data_path + 'olist_order_reviews_dataset.csv')
orders_df = pd.read_csv(data_path + 'olist_orders_dataset.csv')
products_df = pd.read_csv(data_path + 'olist_products_dataset.csv')
sellers_df = pd.read_csv(data_path + 'olist_sellers_dataset.csv')
category_translation_df = pd.read_csv(data_path + 'product_category_name_translation.csv')

df_lists.append(customers_df)
df_lists.append(geolocation_df)
df_lists.append(order_items_df)
df_lists.append(order_payments_df)
df_lists.append(order_reviews_df)
df_lists.append(orders_df)
df_lists.append(products_df)
df_lists.append(sellers_df)
df_lists.append(category_translation_df)

df_names = ['customers', 'geolocation', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers', 'category_translation']

### Obtener información de los data frames

In [27]:
for data in df_lists:
    print(data.info())
    print(data.describe())
    print(data.isnull().sum())
    print(data.nunique())
    print(data.columns)
    print(data.shape)
    print('\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
       customer_zip_code_prefix
count              99441.000000
mean               35137.474583
std                29797.938996
min                 1003.000000
25%                11347.000000
50%                24416.000000
75%                58900.000000
max                99990.000000
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
customer

### Obtener las columnas de los datasets

In [28]:
j = 0
for i in df_lists:
    print(f"{df_names[j]} -> columnas: {i.columns.to_list()}")
    j+=1

customers -> columnas: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
geolocation -> columnas: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']
order_items -> columnas: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']
order_payments -> columnas: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
order_reviews -> columnas: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']
orders -> columnas: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
products -> columnas: ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_le

Texto explicando y analizando las relaciones entre data frames para hacer "joins" más adelante

### Resúmenes Estadísticos de los datos

In [29]:
#Código para graficar distinta informaciónn de los datasets

### Visualización de Patrones Clave

In [30]:
#Código para visualizar patrones clave

### Análisis de correlación de características

In [31]:
# Código para visualizar correlación entre carecterísitcas

## **Dataset final**

In [32]:
# Código para juntar los datasets en uno solo basándose en los campos vistos anteriormente
# Analizar la información por si es necesario eliminar, renombrar o cambiar cualquier columna.
# Dejar bien balanceado el dataset y demostrar que esté preprocesado correctamente.